<a href="https://colab.research.google.com/github/leonardoub/SCRIPT_PALERMO/blob/master/Hyperparameter_optimization_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Optimization of hyperparameter

In [1]:
!pip install -U keras-tuner

     |████████████████████████████████| 92kB 2.8MB/s 
     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 3.8MB 37.9MB/s 
     |████████████████████████████████| 450kB 55.3MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15354 sha256=003bd9dad3f4d433ff3f26a10eaa258b3a54d8e7b340804d3c97ece1c026de43
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables
ERROR: tensorflow 2.1.0rc2 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.3.3 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    U

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

#Load data

In [3]:
#load data from Drive
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
train_dataset_path = '/gdrive/My Drive/AIM_PA/database_training2.csv'
test_dataset_path = '/gdrive/My Drive/AIM_PA/database_nostro_without_nan.csv'

In [0]:
df_train = pd.read_csv(train_dataset_path)
df_test = pd.read_csv(test_dataset_path)

In [0]:
df_train.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_test.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_train.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)
df_test.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)

In [0]:
train_data = df_train.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
test_data = df_test.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
train_labels = df_train.Histology

In [0]:
test_labels = df_test.Histology

##Z score dei dati

In [0]:
mean = train_data.mean(axis=0)
train_data_stand = train_data - mean
std = train_data.std(axis=0)
train_data_stand /= std

In [0]:
test_data_stand = test_data - mean
test_data_stand /= std

##Vettorizzare i label

In [0]:
word_index={'adenocarcinoma':0, 'large cell':1, 'squamous cell carcinoma':2}

In [0]:
train_labels_dec = [word_index[label] for label in train_labels]

In [0]:
test_labels_dec = [word_index[label] for label in test_labels]

In [18]:
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [0]:
one_hot_train_labels = to_categorical(train_labels_dec)
one_hot_test_labels = to_categorical(test_labels_dec)

#PCA

In [0]:
from sklearn.decomposition import PCA

In [0]:
pca = PCA(n_components=0.9, svd_solver='full')

In [22]:
pca.fit(train_data_stand)

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [0]:
train_data_stand_pca = pca.transform(train_data_stand)
test_data_stand_pca = pca.transform(test_data_stand)

In [24]:
train_data_stand_pca.shape

(131, 9)

#Building Network

In [0]:
from tensorflow import keras

In [0]:
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence

In [0]:
from tensorflow.keras import layers

In [0]:
from tensorflow.keras import regularizers

In [0]:
from tensorflow.keras.optimizers import SGD

In [0]:
from kerastuner.tuners import RandomSearch

In [0]:
def build_model(hp):
  model = keras.models.Sequential()
  model.add(layers.Dense(units=(hp.Int('units', min_value=3, max_value=8, step=1)), 
                         activation='relu', input_shape=(9,)))
  model.add(layers.Dense(3, activation='softmax'))
  sgd = SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)
  model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

#Stratified k-fold

This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

In [0]:
from sklearn.model_selection import StratifiedKFold

In [33]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
skf.get_n_splits(train_data_stand_pca, train_labels_dec)

3

In [34]:
for train_index, test_index in skf.split(train_data_stand_pca, train_labels_dec):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [  0   1   4   5   8   9  11  12  14  15  16  17  19  20  22  23  24  25
  27  29  30  33  34  36  37  38  39  40  41  42  44  45  46  48  51  52
  53  56  57  58  59  60  62  63  65  66  67  69  72  76  77  78  79  80
  81  83  84  85  87  88  89  90  92  96  97  98 100 101 102 103 104 105
 107 109 110 111 113 115 117 120 121 122 124 125 127 128] TEST: [  2   3   6   7  10  13  18  21  26  28  31  32  35  43  47  49  50  54
  55  61  64  68  70  71  73  74  75  82  86  91  93  94  95  99 106 108
 112 114 116 118 119 123 126 129 130]
TRAIN: [  2   3   5   6   7   8   9  10  11  12  13  18  20  21  25  26  27  28
  29  30  31  32  34  35  36  38  39  43  44  45  46  47  48  49  50  53
  54  55  57  58  61  63  64  65  66  68  70  71  73  74  75  76  78  82
  84  85  86  87  90  91  92  93  94  95  96  99 100 101 102 105 106 108
 109 111 112 114 115 116 118 119 122 123 124 125 126 127 129 130] TEST: [  0   1   4  14  15  16  17  19  22  23  24  33  37  40  41  42  51  52
  56  59 

In [35]:
train_labels_dec[125]

2

In [0]:
from keras.utils import to_categorical


In [0]:
#for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
#  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
#  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])

#  val_data = np.array([train_data_stand_pca[i] for i in val_index])
#  val_targets = np.array([train_labels_dec[i] for i in val_index])

#  one_hot_partial_train_targets = to_categorical(partial_train_targets)
#  one_hot_val_targets = to_categorical(val_targets)

#  model = build_model()
#  model.fit(partial_train_data, one_hot_partial_train_targets, epochs = num_epochs, batch_size=1)

#  val_loss, val_accuracy = model.evaluate(val_data, one_hot_val_targets)
#  all_scores.append(val_accuracy)
#I parametri per la valutazione vengono calcolati una volta per ogni k-fold, per ogni set di validazione, quindi k volte

C'è un problema: keras.utils.to_categorical produces a one-hot encoded class vector, i.e. the multilabel-indicator mentioned in the error message. StratifiedKFold is not designed to work with such input; i.e. your y must be a 1-D array of your class labels.
Essentially, what you have to do is simply to invert the order of the operations: split first (using your intial y_train), and convert to_categorical afterwards.

In [38]:
num_epochs = 50
all_acc_histories = []
all_loss_histories = []
all_val_acc_histories = []
all_val_loss_histories = []

for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])
  
  val_data = np.array([train_data_stand_pca[i] for i in val_index])
  val_targets = np.array([train_labels_dec[i] for i in val_index])

  one_hot_partial_train_targets = to_categorical(partial_train_targets)
  one_hot_val_targets = to_categorical(val_targets)

 
  tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=5, 
                       executions_per_trial=3, directory='/content/my_dir', project_name='helloworld')
  
  tuner.search_space_summary()

  tuner.search(partial_train_data, one_hot_partial_train_targets, validation_data=(val_data, one_hot_val_targets), 
                      epochs=num_epochs, batch_size=10)
  

#  acc_history = history.history['acc']
#  all_acc_histories.append(acc_history)

#  loss_history = history.history['loss']
#  all_loss_histories.append(loss_history)

#  acc_val_history = history.history['val_acc']
#  all_val_acc_histories.append(acc_val_history)

#  loss_val_history = history.history['val_loss']
#  all_val_loss_histories.append(loss_val_history)
  

#I parametri per la valutazione vengono calcolati per ogni epoca, quindi num_epochs volte. 
#Il tutto viene ripetuto un numero di volte pari a n_splits.
#Si ottiene una lista con n_splits elementi ciascuno dei quali è una lista lunga num_epochs,
#ogni elemento può essere uno fra questi: dict_keys(['val_loss', 'val_acc', 'loss', 'acc']) 

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 4ms/sample - loss: 2.1728 - accuracy: 0.5000 - val_loss: 2.4077 - val_accuracy: 0.3111
Epoch 2/50
86/86 [==============================] - 0s 436us/sample - loss: 2.0917 - accuracy: 0.5000 - val_loss: 2.3247 - val_accuracy: 0.3111
Epoch 3/50
86/86 [==============================] - 0s 374us/sample - loss: 2.0118 - accuracy: 0.5000 - val_loss: 2.2488 - val_accuracy: 0.3111
Epoch 4/50
86/86 [==============================] - 0s 379us/sample - loss: 1.9415 - accuracy: 0.5116 - val_loss: 2.1745 - val_accuracy: 0.3111
Epoch 5/50
86/86 [==============================] - 0s 416us/sample - loss: 1.8734 - accuracy: 0.5116 - val_loss: 2.1077 - val_accuracy: 0.2889
Epoch 6/50
86/86 [==============================] - 0s 410us/sample - loss: 1.8121 - accuracy: 0.5000 - val_loss: 2.0476 - val_accuracy: 0.2889
Epoch 7/50
86/86 [==============================] - 0s 404us/sample - loss: 1.7569 - accuracy:

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 4ms/sample - loss: 2.1155 - accuracy: 0.3372 - val_loss: 1.6294 - val_accuracy: 0.3333
Epoch 2/50
86/86 [==============================] - 0s 381us/sample - loss: 2.0154 - accuracy: 0.3256 - val_loss: 1.5843 - val_accuracy: 0.3333
Epoch 3/50
86/86 [==============================] - 0s 392us/sample - loss: 1.9289 - accuracy: 0.3256 - val_loss: 1.5428 - val_accuracy: 0.3333
Epoch 4/50
86/86 [==============================] - 0s 366us/sample - loss: 1.8520 - accuracy: 0.3605 - val_loss: 1.5052 - val_accuracy: 0.3333
Epoch 5/50
86/86 [==============================] - 0s 379us/sample - loss: 1.7864 - accuracy: 0.3488 - val_loss: 1.4737 - val_accuracy: 0.3333
Epoch 6/50
86/86 [==============================] - 0s 418us/sample - loss: 1.7311 - accuracy: 0.3721 - val_loss: 1.4448 - val_accuracy: 0.3333
Epoch 7/50
86/86 [==============================] - 0s 403us/sample - loss: 1.6801 - accuracy:

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 4ms/sample - loss: 2.7649 - accuracy: 0.2442 - val_loss: 2.0909 - val_accuracy: 0.4000
Epoch 2/50
86/86 [==============================] - 0s 469us/sample - loss: 2.5993 - accuracy: 0.2442 - val_loss: 1.9986 - val_accuracy: 0.4222
Epoch 3/50
86/86 [==============================] - 0s 384us/sample - loss: 2.4388 - accuracy: 0.2558 - val_loss: 1.9104 - val_accuracy: 0.4222
Epoch 4/50
86/86 [==============================] - 0s 357us/sample - loss: 2.2901 - accuracy: 0.2558 - val_loss: 1.8302 - val_accuracy: 0.4000
Epoch 5/50
86/86 [==============================] - 0s 394us/sample - loss: 2.1534 - accuracy: 0.2674 - val_loss: 1.7520 - val_accuracy: 0.4222
Epoch 6/50
86/86 [==============================] - 0s 405us/sample - loss: 2.0297 - accuracy: 0.2791 - val_loss: 1.6833 - val_accuracy: 0.4222
Epoch 7/50
86/86 [==============================] - 0s 377us/sample - loss: 1.9140 - accuracy:

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 4ms/sample - loss: 2.2098 - accuracy: 0.3372 - val_loss: 2.4400 - val_accuracy: 0.3778
Epoch 2/50
86/86 [==============================] - 0s 424us/sample - loss: 2.0920 - accuracy: 0.3372 - val_loss: 2.3508 - val_accuracy: 0.3556
Epoch 3/50
86/86 [==============================] - 0s 398us/sample - loss: 1.9904 - accuracy: 0.3372 - val_loss: 2.2669 - val_accuracy: 0.3556
Epoch 4/50
86/86 [==============================] - 0s 378us/sample - loss: 1.9018 - accuracy: 0.3488 - val_loss: 2.1892 - val_accuracy: 0.3556
Epoch 5/50
86/86 [==============================] - 0s 382us/sample - loss: 1.8216 - accuracy: 0.3721 - val_loss: 2.1273 - val_accuracy: 0.3333
Epoch 6/50
86/86 [==============================] - 0s 378us/sample - loss: 1.7538 - accuracy: 0.3721 - val_loss: 2.0759 - val_accuracy: 0.3333
Epoch 7/50
86/86 [==============================] - 0s 368us/sample - loss: 1.7013 - accuracy:

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 4ms/sample - loss: 2.2588 - accuracy: 0.4302 - val_loss: 1.8225 - val_accuracy: 0.4222
Epoch 2/50
86/86 [==============================] - 0s 411us/sample - loss: 2.0413 - accuracy: 0.4186 - val_loss: 1.7223 - val_accuracy: 0.4000
Epoch 3/50
86/86 [==============================] - 0s 408us/sample - loss: 1.8559 - accuracy: 0.4302 - val_loss: 1.6463 - val_accuracy: 0.4222
Epoch 4/50
86/86 [==============================] - 0s 378us/sample - loss: 1.7094 - accuracy: 0.4767 - val_loss: 1.5889 - val_accuracy: 0.4222
Epoch 5/50
86/86 [==============================] - 0s 428us/sample - loss: 1.6117 - accuracy: 0.4651 - val_loss: 1.5507 - val_accuracy: 0.4000
Epoch 6/50
86/86 [==============================] - 0s 406us/sample - loss: 1.5325 - accuracy: 0.4884 - val_loss: 1.5236 - val_accuracy: 0.4222
Epoch 7/50
86/86 [==============================] - 0s 433us/sample - loss: 1.4777 - accuracy:

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Reloading Oracle from /content/my_dir/helloworld/oracle.json
INFO:tensorflow:Reloading Tuner from /content/my_dir/helloworld/tuner0.json


INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Reloading Oracle from /content/my_dir/helloworld/oracle.json
INFO:tensorflow:Reloading Tuner from /content/my_dir/helloworld/tuner0.json


INFO:tensorflow:Oracle triggered exit


In [41]:
tuner.search_space_summary()


In [0]:
#history_dict = history.history

In [0]:
#history_dict.keys()

In [0]:
#len(all_acc_histories[2])

In [0]:
#average_acc_history = [np.mean([x[i] for x in all_acc_histories]) for i in range(num_epochs)]
##media per epoca degli score ottenuti per tutte le k-fold
##per ogni k-fold di fanno num_epoch epoche, la media viene fatta prendendo gli score di tutti i k-fold relativi ad una data epoca,
##e si fa questo per tutte le epoche
#average_loss_history = [np.mean([x[i] for x in all_loss_histories]) for i in range(num_epochs)]
#average_val_acc_history = [np.mean([x[i] for x in all_val_acc_histories]) for i in range(num_epochs)]
#average_val_loss_history = [np.mean([x[i] for x in all_val_loss_histories]) for i in range(num_epochs)]


In [0]:
#len(average_val_acc_history)

##Plotting training and validation loss

In [0]:
import matplotlib.pyplot as plt

In [0]:
epochs = range(1, num_epochs+1)

In [0]:
plt.plot(epochs, average_loss_history, 'bo', label='training loss')
plt.plot(epochs, average_val_loss_history, 'b', label='validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

##Plotting train and validation accuracy

In [0]:
plt.plot(epochs, average_acc_history, 'bo', label='Training accuracy')
plt.plot(epochs, average_val_acc_history, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend() 

In [0]:
!pwd

In [0]:
!pip list